In [ ]:
from transformers import AutoModelForQuestionAnswering, AutoModelForSeq2SeqLM, AutoTokenizer, DebertaV2TokenizerFast, pipeline
from sentence_transformers import SentenceTransformer
import torch
import numpy as np
import pandas as pd
import re
import os
import faiss
import pydantic
from abc import ABC
from typing import Tuple, List, Dict, Optional, Any
from sklearn.preprocessing import normalize
from peft import PeftModel, PeftConfig, PeftModelForQuestionAnswering
from langchain.chains import RetrievalQA, ConversationalRetrievalChain, StuffDocumentsChain, LLMChain
from langchain.chains.base import Chain
from langchain.memory import ConversationBufferMemory
from langchain.prompts import PromptTemplate, BasePromptTemplate, SystemMessagePromptTemplate
from langchain.prompts.base import BasePromptTemplate
from langchain.llms import BaseLLM, HuggingFacePipeline
from langchain.agents import Tool, AgentExecutor, ZeroShotAgent
from langchain.utilities import WikipediaAPIWrapper
from langchain.tools import WikipediaQueryRun
from langchain_community.vectorstores import FAISS
from langchain_core.retrievers import BaseRetriever
from langchain_core.stores import InMemoryStore
from langchain_core.documents import Document
from langchain_core.runnables import Runnable
from langchain_core.outputs import Generation, LLMResult
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.chat_models import ChatOpenAI      # Agent LLM API 활용할 때만 활성화 (비상시)
# from google.colab import userdata                           # Agent LLM API 활용할 때만 활성화 (비상시)
from dotenv import load_dotenv                              # Agent LLM API 활용할 때만 활성화 (비상시)
load_dotenv()                                               # Agent LLM API 활용할 때만 활성화 (비상시)

True

In [3]:
# ✅ 병렬 토크나이저 경고 방지
os.environ["TOKENIZERS_PARALLELISM"] = "false"

# ✅ 디바이스 설정
device = "cuda" if torch.cuda.is_available() else "cpu"

# ✅ Q-LoRA 설정 로드
peft_config = PeftConfig.from_pretrained("./trained_V3_LoRA")

# ✅ 기본 모델 로드
loaded_model = AutoModelForQuestionAnswering.from_pretrained(peft_config.base_model_name_or_path)

# ✅ Q-LoRA 어댑터 로드
loaded_model = PeftModel.from_pretrained(loaded_model, "./trained_V3_LoRA")

# ✅ 학습된 Q-LoRA 모델 및 토크나이저 로드
loaded_model = loaded_model.to(device)
loaded_tokenizer = AutoTokenizer.from_pretrained("./trained_V3_LoRA")

# ✅ 문장 임베딩 모델 로드 (LangChain 호환)
embedding_model = HuggingFaceEmbeddings(model_name="intfloat/multilingual-e5-large")  # snunlp/KR-SBERT-V40K-klueNLI-augSTS

# ✅ 1️⃣ 데이터 로드 (박물관 데이터)
data_path = './data/museum_data_rhys_250326.csv'
df = pd.read_csv(data_path)
df = df[["Title", "Description"]].dropna().rename(columns={"Title": "question", "Description": "answer"})

In [4]:
# ✅ 불용문자 제거 함수 (숫자, 영어, 한국어 자음/모음, 특수문자 제거)
def remove_stopwords(text):
    text = re.sub(r"[^0-9a-zA-Zㄱ-ㅎㅏ-ㅣ\w\s]", "", text)
    return text.strip()

# ✅ 2️⃣ DataFrame의 Title에 대한 임베딩 생성 및 L2 정규화
title_embeddings = embedding_model.embed_documents(df['question'].tolist())
title_embeddings_normalized = normalize(title_embeddings, axis=1, norm='l2')

# ✅ FAISS 인덱스 생성 및 임베딩 저장 (Inner Product 기반)
index = faiss.IndexFlatIP(len(title_embeddings[0]))
index.add(title_embeddings_normalized.astype('float32'))

# ✅ 문장 분할 함수 (간단한 마침표, 물음표, 느낌표 기준 + 공백 제거)
def split_sentences(text):
    sentences = re.split(r'(?<=[.?!])\s+', text.strip())
    return [s for s in sentences if s]

# ✅ 제목과 그에 해당하는 문장들을 매핑하는 딕셔너리 생성
title_to_sentences = {}
for _, row in df.iterrows():
    title = row["question"]
    description = row["answer"]
    sentences = split_sentences(description)
    title_to_sentences[title] = sentences

# ✅ 사용자 정의 retriever 클래스
class CustomTitleSentenceRetriever(BaseRetriever):
    def get_relevant_documents(self, query):
        # 질문에 대한 임베딩 생성 및 정규화
        question_embedding = embedding_model.embed_query(remove_stopwords(query))
        question_embedding_normalized = normalize(np.array([question_embedding]), axis=1, norm='l2')

        # FAISS 인덱스에서 유사한 제목 검색
        distances, indices = index.search(question_embedding_normalized, 10)    # top_n=10 대신 10을 위치 인자로 전달
        similar_titles = df['question'].iloc[indices[0]].tolist()

        # 유사한 제목에 해당하는 문장들을 Document 형태로 변환하여 반환
        retrieved_documents = []
        for title in similar_titles:
            if title in title_to_sentences:
                for sentence in title_to_sentences[title]:
                    retrieved_documents.append(Document(page_content=sentence, metadata={"title": title}))

        return retrieved_documents

# ✅ 사용자 정의 retriever 인스턴스 생성
custom_retriever = CustomTitleSentenceRetriever()

C:\Users\Playdata\AppData\Local\Temp\ipykernel_21772\3763482784.py:28: DeprecationWarning: Retrievers must implement abstract `_get_relevant_documents` method instead of `get_relevant_documents`
  class CustomTitleSentenceRetriever(BaseRetriever):


In [5]:
# ✅ 3️⃣ Wikipedia 검색 도구 설정
wiki_api = WikipediaAPIWrapper(lang="ko")
wikipedia_tool = WikipediaQueryRun(api_wrapper=wiki_api)

# ✅ 4️⃣ 박물관 데이터 검색 함수 (유사도 기반 정보검색)
def search_museum_data_semantic(input_question, top_n=10):

    print(f"search_museum_data_semantic - input_question: {input_question}, type: {type(input_question)}")          # 디버깅을 위한 출력 코드

    # 질문에 대한 임베딩 생성 및 정규화
    cleaned_question = remove_stopwords(input_question)

    print(f"search_museum_data_semantic - cleaned_question: {cleaned_question}, type: {type(cleaned_question)}")    # 디버깅을 위한 출력 코드

    question_embedding = embedding_model.embed_query(cleaned_question)
    question_embedding_normalized = normalize(np.array([question_embedding]), axis=1, norm='l2')

    # FAISS 인덱스에서 유사한 임베딩 검색
    distances, indices = index.search(question_embedding_normalized, top_n)

    # 검색 결과에 해당하는 문서 추출
    results = df.iloc[indices[0]]
    top_results = results['answer'].tolist() # 상위 5개 검색 결과를 변수에 저장

    print(f"Museum Data Search Output: {top_results}, Type: {type(top_results)}")   # 디버깅을 위한 출력 코드
    return top_results # 변수 반환

# ✅ 5️⃣ 도구 목록 정의 (Agent가 사용할 도구)
museum_tool = Tool(
    name="Museum Data Search",
    func=lambda q: "\n\n".join(search_museum_data_semantic(q)),     # 반환 값을 문자열로 변환
    description="This is useful when you need to answer questions about even the slightest of relevant information from the National Museum of Korea. Try searching first in the museum database. You need to input about even the slightest of relevant information from the museum. If you cannot find the appropriate answer in the database, try searching for Wikipedia. If you did not ask a question about the even the slightest of relevant information from the museum, you should specify to the user that it is not an appropriate question."
)
wiki_tool = Tool(
    name="Wikipedia Search",
    func=lambda q: "\n\n".join(wikipedia_tool.run(q)),              # 반환 값을 문자열로 변환
    description="This is useful if you have not found an appropriate answer to a user's question about the even the slightest of relevant information in the database. If you did not ask a question about the even the slightest of relevant information from the museum, you should specify to the user that it is not an appropriate question."
)
tools = [museum_tool, wiki_tool]

# ✅ 6️⃣ 메모리 설정
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

C:\Users\Playdata\AppData\Local\Temp\ipykernel_21772\3386679428.py:42: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)


In [ ]:
# ✅ 7️⃣ Agent에 사용할 LLM을 "google/umt5-small"로 정의 (AutoModelForSeq2SeqLM으로 호출)
# control_model_id = "google/umt5-small"
# control_tokenizer = AutoTokenizer.from_pretrained(control_model_id)
# control_model = AutoModelForSeq2SeqLM.from_pretrained(control_model_id)

# ✅ Langchain의 HuggingFacePipeline으로 "google/umt5-small"을 래핑
# llm = HuggingFacePipeline(
#     pipeline=pipeline(
#         "text2text-generation",
#         model=control_model,
#         tokenizer=control_tokenizer,
#         device=device,
#         max_length=2048,        # 적절한 max_length 설정
#         max_new_tokens=1024,    # 적절한 max_new_tokens 설정
#         temperature=0.3,
#         top_p=0.7,
#         repetition_penalty=1.2,
#         model_kwargs={"do_sample": True}
#     )
# )

# ✅ Agent에 사용할 표준 LLM 코드 (gpt-4o-omni 사용)
openai_api_key = os.environ.get("OPENAI_API_KEY")
# openai_api_key = userdata.get("OPENAI_API_KEY")
llm = ChatOpenAI(
            api_key=openai_api_key,
            model_name="gpt-4o-omni",           # gpt-4o-mini 대신 선택
            temperature=0.3,
            max_tokens=2048
)

C:\Users\Playdata\AppData\Local\Temp\ipykernel_21772\2045584064.py:25: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm = ChatOpenAI(


In [7]:
# ✅ 9️⃣ Agent 프롬프트 템플릿
prefix = """You are a useful agent designed to answer even the slightest of relevant questions about the National Museum of Korea. You have to find the words that match the information in the museum database from the user's questions and generate answers. To do this, you can access the following tools. Carefully review your questions and descriptions of the tools available to determine which tools are most appropriate to use. You have to try using museum_tool first. If you can't find any the slightest of relevant information in museum_tool, you should use wiki_tool. If you need a tool, clarify which tools you will use and provide accurate inputs for that tool. If you are able to answer questions without using the tool, please provide a direct answer.

Available tools:"""
suffix = """Previous conversation: {chat_history}

Begin!

Question: {input}

{agent_scratchpad}"""

prompt = ZeroShotAgent.create_prompt(
    tools,
    prefix=prefix,
    suffix=suffix,
    input_variables=["input", "chat_history", "agent_scratchpad"]
)

# ✅ 1️⃣0️⃣ LLMChain 생성
llm_chain = LLMChain(llm=llm, prompt=prompt)

# ✅ 1️⃣1️⃣ Agent 생성
agent = ZeroShotAgent(llm_chain=llm_chain, tools=tools, verbose=True)       # verbose=True로 설정하면 Agent의 사고 과정을 볼 수 있습니다.

# ✅ 1️⃣2️⃣ AgentExecutor 생성
agent_executor = AgentExecutor.from_agent_and_tools(
    agent=agent, tools=tools, verbose=True, memory=memory, handle_parsing_errors=True
)

C:\Users\Playdata\AppData\Local\Temp\ipykernel_21772\2351043044.py:21: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  llm_chain = LLMChain(llm=llm, prompt=prompt)
C:\Users\Playdata\AppData\Local\Temp\ipykernel_21772\2351043044.py:24: LangChainDeprecationWarning: LangChain agents will continue to be supported, but it is recommended for new use cases to be built with LangGraph. LangGraph offers a more flexible and full-featured framework for building agents, including support for tool-calling, persistence of state, and human-in-the-loop workflows. For details, refer to the `LangGraph documentation <https://langchain-ai.github.io/langgraph/>`_ as well as guides for `Migrating from AgentExecutor <https://python.langchain.com/docs/how_to/migrate_agent/>`_ and LangGraph's `Pre-built ReAct agent <https://langchain-ai.github.io/langgraph/how-tos/create-react-agent/>`

In [8]:
# ✅ 1️⃣3️⃣ 사용자 정의 LLM 래퍼 (파인튜닝된 모델 통합) - ConversationalRetrievalChain에 사용
class CustomQAmodel(BaseLLM, Runnable):
    model: PeftModelForQuestionAnswering        # 타입 힌트 객체 지정
    tokenizer: DebertaV2TokenizerFast           # 타입 힌트 객체 지정
    device: str

    @property
    def _llm_type(self) -> str:
        return "custom_qa_model"

    def _generate(self, prompts: List[str], stop: Optional[List[str]] = None) -> LLMResult:
        print("Test inside generate")                   # 모델 프롬프트 형태를 확인하기 위해 추가
        generations = []
        for prompt in prompts:      # 입력된 모든 프롬프트에 대해 반복 처리
            print(f"CustomQAmodel Prompt: {prompt}")    # 모델 프롬프트 형태를 확인하기 위해 추가
            try:
                context_match = re.search(r"Context:\n(.*?)\nQuestion:", prompt, re.DOTALL)
                question_match = re.search(r"Question:\n(.*?)(?:\nPrevious conversation:(.*?))?\nAnswer:", prompt, re.DOTALL)

                if not context_match or not question_match:
                    generations.append([Generation(text="unknown")])            # 또는 "not sure" 등 메인 루프에서 감지할 수 있는 키워드
                    continue

                context_part = context_match.group(1).strip()
                print(f"CustomQAmodel Context Part: {context_part}")            # 추출된 내용 확인을 위한 디버깅 코드

                question_part = question_match.group(1).strip() if question_match.group(1) else question_match.group(3).strip()
                print(f"CustomQAmodel Question Part: {question_part}")          # 추출된 내용 확인을 위한 디버깅 코드

                chat_history_part = question_match.group(2).strip() if question_match.group(2) else (question_match.group(4).strip() if question_match.group(4) else "")
                print(f"CustomQAmodel Chat History Part: {chat_history_part}")  # 추출된 내용 확인을 위한 디버깅 코드

            except Exception as e:
                generations.append([Generation(text=f"Error parsing prompt in CustomQAmodel: {e}")])
                continue

            # 추출한 chat_history를 모델 입력에 포함하는 방식 결정
            # 예시: context와 question 앞에 chat_history를 추가하여 입력
            augmented_input = f"{chat_history_part}\n{context_part}\n{question_part}"

            inputs = self.tokenizer(
                augmented_input,
                truncation="only_second",
                max_length=3072,
                padding="max_length",
                return_tensors="pt"
            ).to(self.device)

            print(f"CustomQAmodel Inputs: {inputs}")        # 모델 입력 형태를 확인하기 위해 추가

            with torch.no_grad():
                outputs = self.model(**inputs)

            print(f"CustomQAmodel Outputs: {outputs}")      # 모델 출력 형태를 확인하기 위해 추가

            answer_start_scores = outputs.start_logits
            answer_end_scores = outputs.end_logits
            answer_start = torch.argmax(answer_start_scores)
            answer_end = torch.argmax(answer_end_scores) + 1
            answer = self.tokenizer.convert_tokens_to_string(self.tokenizer.convert_ids_to_tokens(inputs["input_ids"][0][answer_start:answer_end]))

            print(f"CustomQAmodel Answer: {answer}")        # 모델 정답 형태를 확인하기 위해 추가

            generations.append([Generation(text=answer.replace("[CLS]", "").replace("[SEP]", "").strip())])

            print(f"CustomQAmodel Final Prompt: {prompt}")  # 모델 프롬프트 형태를 확인하기 위해 추가

        return LLMResult(generations=generations)

    def _call(self, prompt: str, stop: Optional[List[str]] = None) -> str:
        print(f"CustomQAmodel Prompt (inside _call): {prompt}") # 모델 프롬프트 형태를 확인하기 위해 추가
        return self._generate([prompt], stop=stop)[0].text

In [9]:
# ✅ 1️⃣4️⃣ 프롬프트 템플릿 (LangChain) - ConversationalRetrievalChain을 사용
prompt_template = """{prompt}

Context: {context}

Question: {question}

Previous conversation: {chat_history}

Answer:"""

PROMPT = PromptTemplate(template=prompt_template, input_variables=["context", "question", "chat_history", "prompt"])

system_rules = """

<SYSTEM_RULE>

1. NEVER mention that you're an AI. You are rather going to play a role as a docent of the National Museum of Korea in South Korea. You have to find the words that match the information in the museum's database from the user's questions, generate answers, and politely explain them to the user.

2. Sense which country's language is the language of the user's question and you have to answer it in the language of the same country.

3. Refrain from disclaimers about you not being a professional or expert.

4. Keep responses unique and free of repetition.

5. Answer based on the relevant information learned. If the relevant information is insufficient or does not exist in the file, please refer to RAG (search-based response) for reliable information.

6. Always focus on the key points in user's questions to determine user's intent.

7. Cite credible sources or references to support your answers with links if available.

8. If a mistake is made in a previous response, recognize and correct it.

9. If you don't have enough evidence or are unsure of the information, state “unknown” or “not sure” instead of making it up.

10. Use only the information you know for certain to create a concise answer. If you have to make a guess, say “this is a guess”.

"""

# ✅ 1️⃣5️⃣ ConversationalRetrievalChain 설정
qa = ConversationalRetrievalChain.from_llm(
    llm=CustomQAmodel(model=loaded_model, tokenizer=loaded_tokenizer, device=device),
    retriever=custom_retriever, # 사용자 정의 Langchain retriever 인스턴스 사용
    memory=memory,
    chain_type="stuff",
    condense_question_llm=llm,  # 질문 재구성을 위한 LLM 설정
    combine_docs_chain_kwargs={"prompt": PROMPT.partial(prompt=system_rules)}
)

In [ ]:
# ✅ 1️⃣6️⃣ 답변 생성 단락
while True:
    print("While loop is running")                                  # 추출된 내용 확인을 위한 디버깅 코드
    question = input("궁금한 점을 질문해주세요 (종료하려면 'exit' 입력): ")
    if question.lower() == 'exit':
        print("도슨트 서비스를 종료합니다. 이용해주셔서 고맙습니다.")
        break

    # ConversationalRetrievalChain을 사용하여 답변 시도
    print("Invoking ConversationalRetrievalChain...")               # 추출된 내용 확인을 위한 디버깅 코드
    result_qa = qa.invoke({"question": question})
    answer = result_qa["answer"]

    # ✅ context_part, question_part, chat_history_part 변수를 CustomQAmodel 내부에서 가져오기
    context_part = result_qa.get("context_part", "Not Found")  # context_part가 없으면 "Not Found" 출력
    question_part = result_qa.get("question_part", "Not Found")  # question_part가 없으면 "Not Found" 출력
    chat_history_part = result_qa.get("chat_history_part", "Not Found")  # chat_history_part가 없으면 "Not Found" 출력
    print(f"CustomQAmodel Context Part: {context_part}")            # 추출된 내용 확인을 위한 디버깅 코드
    print(f"CustomQAmodel Question Part: {question_part}")          # 추출된 내용 확인을 위한 디버깅 코드
    print(f"CustomQAmodel Chat History Part: {chat_history_part}")  # 추출된 내용 확인을 위한 디버깅 코드

    # ✅ query / question / answer 출력 문제 파악을 위한 디버깅 코드
    print(f"prompt: {prompt}")                                  # 모델 출력 형태를 확인하기 위해 추가
    print(f"qa: {qa}")                                          # 모델 출력 형태를 확인하기 위해 추가
    print(f"qa.input_keys: {qa.input_keys}")                    # 모델 출력 형태를 확인하기 위해 추가
    print(f"result_qa: {result_qa}")                            # 모델 출력 형태를 확인하기 위해 추가
    print(f"answer.split(): {answer.split()}")                  # 모델 출력 형태를 확인하기 위해 추가
    print(f"len(answer.split()): {len(answer.split())}")        # 모델 출력 형태를 확인하기 위해 추가

    print(f"질문 (박물관 데이터): {question}")
    print(f"답변 (박물관 데이터): {answer}")

    # 박물관 데이터에서 맥락을 찾지 못한 경우 (답변이 부실하거나 특정 키워드를 포함하는 경우), Agent 실행
    if len(answer.split()) == 0 or "죄송합니다. 제가 가진 정보로는 질문에 대한 구체적인 답변을 드릴 수 없습니다." in answer or "unknown" in answer.lower() or "not sure" in answer.lower():
        print("박물관 데이터에서는 관련 정보를 찾지 못했습니다. 위키피디아에서 검색을 시도합니다....")
        result_agent = agent_executor.invoke({"input": question})             # Agent 실행 시 chat_history는 메모리에서 관리
        print(f"질문 (위키피디아): {question}")
        print(f"답변 (위키피디아): {result_agent['output']}")
    else:
        print(f"질문 (박물관 데이터 재검색): {question}")
        print(f"답변 (박물관 데이터 재검색): {answer}")

    print("\n")